In [ ]:
####create 
import pandas as pd
import numpy as np
from transformers import LlamaForCausalLM, LlamaTokenizer
from transformers import AutoModel, AutoTokenizer  
from huggingface_hub import login
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig
import json
from unimol_tools import UniMolRepr

2024-12-31 10:36:21 | unimol_tools\weights\weighthub.py | 17 | INFO | Uni-Mol Tools | Weights will be downloaded to default directory: c:\Users\tianren\Anaconda3\lib\site-packages\unimol_tools\weights


In [ ]:
def transform_polymer_json(input_data):
    """Transform the JSON structure to use polymer IDs as keys."""
    transformed_data = {}
    
    # Iterate through polymers and restructure
    for polymer in input_data["polymers"]:
        polymer_id = polymer["id"]
        # Create new structure without the id field since it's now the key
        transformed_data[polymer_id] = {
           # "name": polymer["name"],
            "description": polymer["description"],
            "metadata": polymer["metadata"]
        }
    
    return transformed_data

with open(f"polymer_descriptions.json", 'r') as file:
    polymer_language = json.load(file)
transformed_polymer_language = transform_polymer_json(polymer_language)



In [ ]:
from psmiles import PolymerSmiles as PS
from rdkit import Chem
failed_conformers=0
polymer_conf_embedding_text = {}
unimol_model = UniMolRepr(
               data_type='molecule',
               remove_hs=False,
               model_name='unimolv2',
               model_size='1.1B'
            )

for key, text in sorted(transformed_polymer_language.items()):

    ps = PS(text['metadata']['polymer_smiles'])
    #dimer_smiles = ps.dimer(1)

    dimer_mol = Chem.MolFromSmiles(ps.psmiles)
    for atom in dimer_mol.GetAtoms():
        if atom.GetSymbol() == '*':
            atom.SetAtomicNum(6)  # Convert to carbon
    smiles =  Chem.MolToSmiles(dimer_mol)        
    print(smiles)
    unimol_repr = unimol_model.get_repr([smiles], return_atomic_reprs=True)
    if unimol_repr['cls_repr'][0] is None:
        print(f"Warning: Failed to generate valid conformer for {item['id']}")
        failed_conformers += 1
    conf_embed = np.array(unimol_repr['cls_repr'][0])
    polymer_conf_embedding_text[key] = conf_embed
    

In [9]:
import pickle
with open(f"uni_polymer_embeddings.pickle", "wb") as fp:
    pickle.dump(polymer_conf_embedding_text, fp)